In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Challenge : Construire un Agent RAG Auto-Correcteur avec LangGraph\n",
    "\n",
    "Bienvenue dans ce notebook ! L'objectif est de construire un agent conversationnel sophistiqué en utilisant le framework **LangGraph**. \n",
    "\n",
    "Cet agent ne se contentera pas de répondre à des questions sur la base de documents fournis (le principe du RAG - Retrieval-Augmented Generation), mais il sera capable d'évaluer la pertinence des informations récupérées. Si les informations ne sont pas pertinentes, l'agent **reformulera la question** de l'utilisateur pour tenter d'obtenir de meilleurs résultats, créant ainsi une boucle de raisonnement et d'auto-correction.\n",
    "\n",
    "## Plan du Notebook\n",
    "\n",
    "1.  **Configuration de l'Environnement** : Chargement des clés API et des variables d'environnement.\n",
    "2.  **Construction de la Base de Connaissances (Le \"R\" de RAG)** : Chargement de documents depuis le web, découpage, et création d'un index vectoriel avec ChromaDB.\n",
    "3.  **Définition de l'État de l'Agent** : Création de la structure de données qui servira de \"mémoire\" à notre agent tout au long du processus.\n",
    "4.  **Création des Outils et des Nœuds du Graphe** : Définition des différentes fonctions qui représenteront les étapes de notre agent (recherche, évaluation, réécriture, génération).\n",
    "5.  **Assemblage du Graphe avec LangGraph** : Connexion des nœuds avec une logique conditionnelle pour orchestrer le flux de travail de l'agent.\n",
    "6.  **Compilation et Test** : Création de l'application exécutable et validation de son comportement.\n",
    "7.  **Fonction d'Interface pour Streamlit** : Création d'une fonction `run_agent` simple que notre application Streamlit pourra appeler."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 1 : Configuration de l'Environnement\n",
    "\n",
    "La première étape consiste à s'assurer que notre environnement est correctement configuré. Nous allons charger les bibliothèques nécessaires et les clés API à partir d'un fichier `.env`. C'est une pratique essentielle pour garder nos informations sensibles (comme les clés API) séparées de notre code."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "from dotenv import load_dotenv\n",
    "\n",
    "# Charger les variables d'environnement du fichier .env\n",
    "# Cela permet à os.getenv() de trouver les clés que nous avons stockées.\n",
    "load_dotenv()\n",
    "\n",
    "# Configuration optionnelle pour le traçage avec LangSmith\n",
    "# LangSmith est un outil fantastique pour visualiser et déboguer les chaînes et agents LangChain.\n",
    "# C'est très utile pour comprendre ce qui se passe sous le capot de notre graphe.\n",
    "os.environ[\"LANGCHAIN_TRACING_V2\"] = \"true\"\n",
    "os.environ[\"LANGCHAIN_ENDPOINT\"] = \"https://api.smith.langchain.com\"\n",
    "# Assurez-vous que LANGCHAIN_API_KEY est défini dans votre fichier .env si vous souhaitez utiliser LangSmith\n",
    "\n",
    "# Récupération des clés API\n",
    "GROQ_API_KEY = os.getenv(\"GROQ_API_KEY\")\n",
    "GOOGLE_API_KEY = os.getenv(\"GOOGLE_API_KEY\") # Utilisé pour les embeddings\n",
    "\n",
    "print(\"Clés API chargées.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 2 : Construction de la Base de Connaissances (Le \"R\" de RAG)\n",
    "\n",
    "Un agent RAG a besoin d'une base de connaissances sur laquelle s'appuyer. Nous allons construire cette base en plusieurs sous-étapes :\n",
    "\n",
    "1.  **Chargement des données** : Nous utiliserons `WebBaseLoader` de LangChain pour charger le contenu de quelques articles de blog pertinents sur le Machine Learning et les agents.\n",
    "2.  **Découpage des documents** : Les documents bruts sont souvent trop longs pour être traités efficacement par un LLM. Nous les découperons en plus petits morceaux (chunks) avec `RecursiveCharacterTextSplitter`. Cette méthode est robuste car elle essaie de couper le texte sur des séparateurs logiques (paragraphes, phrases, etc.).\n",
    "3.  **Création des Embeddings** : Pour que la machine comprenne le sens sémantique de nos morceaux de texte, nous devons les convertir en vecteurs numériques (embeddings). Nous utiliserons un modèle d'embedding de Google.\n",
    "4.  **Stockage dans un Vectorstore** : Ces vecteurs seront stockés et indexés dans une base de données vectorielle, **ChromaDB**. Cela nous permettra d'effectuer des recherches de similarité rapides : trouver les morceaux de texte les plus pertinents pour une question donnée.\n",
    "5.  **Création de l'outil de recherche (Retriever Tool)** : Enfin, nous encapsulerons notre logique de recherche dans un `Tool` LangChain. Cela permettra à notre agent d'appeler la recherche dans la base de connaissances comme n'importe quel autre outil."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "from langchain_community.document_loaders import WebBaseLoader\n",
    "from langchain.text_splitter import RecursiveCharacterTextSplitter\n",
    "from langchain_community.vectorstores import Chroma\n",
    "from langchain_google_genai import GoogleGenerativeAIEmbeddings\n",
    "from langchain.tools.retriever import create_retriever_tool\n",
    "\n",
    "# 1. Chargement des données\n",
    "# Nous choisissons des URLs qui parlent des agents et du RAG pour que notre agent soit bien informé sur ce sujet.\n",
    "urls = [\n",
    "    \"https://lilianweng.github.io/posts/2023-06-23-agent/\",\n",
    "    \"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/\",\n",
    "    \"https://lilianweng.github.io/posts/2023-10-25-adv-rag/\",\n",
    "]\n",
    "\n",
    "loader = WebBaseLoader(urls)\n",
    "docs = loader.load()\n",
    "print(f\"{len(docs)} documents chargés.\")\n",
    "\n",
    "# 2. Découpage des documents\n",
    "# chunk_size définit la taille maximale de chaque morceau.\n",
    "# chunk_overlap conserve une petite partie de la fin d'un morceau au début du suivant pour préserver le contexte.\n",
    "text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)\n",
    "splits = text_splitter.split_documents(docs)\n",
    "print(f\"{len(splits)} morceaux créés.\")\n",
    "\n",
    "# 3. Création des Embeddings\n",
    "# Nous utilisons un modèle de Google. Il est performant et bien intégré avec LangChain.\n",
    "embeddings = GoogleGenerativeAIEmbeddings(model=\"models/embedding-001\")\n",
    "\n",
    "# 4. Stockage dans un Vectorstore\n",
    "# Chroma est une base de données vectorielle open-source et légère, parfaite pour des projets comme celui-ci.\n",
    "# \"from_documents\" s'occupe de générer les embeddings et de les stocker.\n",
    "vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)\n",
    "\n",
    "# 5. Création de l'outil de recherche\n",
    "# Nous transformons notre vectorstore en un \"retriever\", qui est un objet capable de récupérer des documents.\n",
    "retriever = vectorstore.as_retriever()\n",
    "\n",
    "# Enfin, nous créons un \"Tool\". L'agent utilisera le nom et la description pour décider quand utiliser cet outil.\n",
    "retriever_tool = create_retriever_tool(\n",
    "    retriever,\n",
    "    name=\"retrieval_tool\",\n",
    "    description=\"Recherche des informations sur les agents d'IA, le RAG et le prompt engineering. Utilise cet outil pour répondre aux questions sur le machine learning.\"\n",
    ")\n",
    "\n",
    "tools = [retriever_tool]\n",
    "print(\"Outil de recherche créé et prêt à l'emploi.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 3 : Définition de l'État de l'Agent\n",
    "\n",
    "LangGraph fonctionne comme une **machine à états**. Nous devons définir explicitement la structure des données qui persisteront et seront modifiées à travers les différents nœuds de notre graphe. C'est ce que nous appelons l'\"État\" (`State`).\n",
    "\n",
    "Notre état, que nous nommerons `AgentState`, contiendra l'historique de la conversation. Nous utilisons `TypedDict` pour définir clairement les champs et leurs types. `Annotated[list[BaseMessage], operator.add]` est une syntaxe spéciale de LangGraph qui indique que le champ `messages` est une liste de messages et que les nouvelles valeurs doivent être ajoutées à la liste existante, plutôt que de la remplacer."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "from typing import TypedDict, Annotated, List\n",
    "from langchain_core.messages import BaseMessage\n",
    "import operator\n",
    "\n",
    "class AgentState(TypedDict):\n",
    "    \"\"\"\n",
    "    Définit la structure de l'état de notre agent.\n",
    "    \n",
    "    Attributes:\n",
    "        messages: L'historique des messages de la conversation.\n",
    "                  L'annotation `operator.add` signifie que les nouveaux messages\n",
    "                  seront ajoutés à la liste existante à chaque étape.\n",
    "    \"\"\"\n",
    "    messages: Annotated[list[BaseMessage], operator.add]\n",
    "\n",
    "print(\"État de l'agent défini.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 4 : Création des Outils et des Nœuds du Graphe\n",
    "\n",
    "C'est le cœur de notre agent. Nous allons définir plusieurs fonctions Python qui serviront de **nœuds** dans notre graphe. Chaque nœud effectue une tâche spécifique.\n",
    "\n",
    "1.  **Modèle LLM** : Nous choisissons le modèle qui servira de \"cerveau\" à notre agent. Nous utilisons `ChatGroq` pour sa rapidité.\n",
    "2.  **Nœud `My_AI_Assistant` (Routeur)** : C'est le point d'entrée. Ce nœud examine la dernière question de l'utilisateur et, en utilisant le LLM et la description des outils disponibles, décide s'il doit appeler un outil (notre `retrieval_tool`) ou s'il peut répondre directement.\n",
    "3.  **Nœud `retrieve` (Action)** : Ce nœud est un `ToolNode`. Il est responsable de l'exécution effective des appels d'outils décidés par le routeur.\n",
    "4.  **Fonction `grade_documents` (Aiguilleur Conditionnel)** : Ce n'est pas un nœud, mais une fonction qui sera utilisée pour créer une **arête conditionnelle**. Elle prend les documents récupérés, demande à un LLM de les noter (sont-ils pertinents ?), et retourne une décision (\"generate\" ou \"rewrite\"). Pour garantir une sortie fiable, nous utilisons les **sorties structurées** de LangChain, forçant le LLM à répondre dans un format JSON que nous définissons.\n",
    "5.  **Nœud `generate` (Générateur de réponse)** : Ce nœud prend les documents jugés pertinents et la question de l'utilisateur, et génère la réponse finale.\n",
    "6.  **Nœud `rewrite` (Réécrivain de question)** : Si les documents ne sont pas pertinents, ce nœud est appelé. Il prend la question originale et demande au LLM de la reformuler pour qu'elle soit plus claire ou plus spécifique, dans l'espoir d'obtenir de meilleurs résultats de recherche."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "from langchain_groq import ChatGroq\n",
    "from langgraph.prebuilt import ToolNode\n",
    "from langchain_core.pydantic_v1 import BaseModel, Field\n",
    "from langchain_core.output_parsers import JsonOutputParser\n",
    "from langchain_core.prompts import PromptTemplate\n",
    "from langchain_core.messages import HumanMessage, ToolMessage\n",
    "\n",
    "# 1. Modèle LLM\n",
    "# Nous utilisons Groq pour sa vitesse d'inférence impressionnante.\n",
    "# Le modèle Llama3 8b est un bon compromis entre performance et rapidité.\n",
    "llm = ChatGroq(model=\"llama3-8b-8192\", temperature=0)\n",
    "\n",
    "# Lier les outils au LLM. Cela permet au LLM de savoir quels outils il peut appeler.\n",
    "llm_with_tools = llm.bind_tools(tools)\n",
    "\n",
    "# 2. Nœud `My_AI_Assistant` (Routeur)\n",
    "def ai_assistant(state: AgentState):\n",
    "    \"\"\"Point d'entrée : décide s'il faut appeler un outil ou non.\"\"\"\n",
    "    print(\"---ASSISTANT (ROUTEUR)---\")\n",
    "    messages = state[\"messages\"]\n",
    "    # Appel au LLM avec la capacité d'utiliser des outils\n",
    "    response = llm_with_tools.invoke(messages)\n",
    "    # La réponse est ajoutée à l'état, qu'elle contienne un appel d'outil ou une réponse directe.\n",
    "    return {\"messages\": [response]}\n",
    "\n",
    "# 3. Nœud `retrieve` (Action)\n",
    "# ToolNode est un nœud pré-construit qui exécute les outils appelés par le LLM.\n",
    "retrieve = ToolNode(tools)\n",
    "\n",
    "# 4. Fonction `grade_documents` (Aiguilleur Conditionnel)\n",
    "\n",
    "# Définition du format de sortie pour l'évaluation\n",
    "class Grade(BaseModel):\n",
    "    \"\"\"Évaluation binaire de la pertinence des documents.\"\"\"\n",
    "    binary_score: str = Field(description=\"Les documents sont-ils pertinents ? 'oui' ou 'non'\")\n",
    "\n",
    "def grade_documents(state: AgentState):\n",
    "    \"\"\"Évalue si les documents récupérés sont pertinents pour la question.\"\"\"\n",
    "    print(\"---ÉVALUATION DES DOCUMENTS---\")\n",
    "    messages = state[\"messages\"]\n",
    "    last_message = messages[-1]\n",
    "    question = messages[0].content\n",
    "\n",
    "    # Créer un parser pour la sortie structurée\n",
    "    parser = JsonOutputParser(pydantic_object=Grade)\n",
    "\n",
    "    # Créer un LLM avec la sortie structurée\n",
    "    structured_llm_grader = llm.with_structured_output(Grade)\n",
    "\n",
    "    # Prompt pour l'évaluation\n",
    "    prompt = PromptTemplate(\n",
    "        template=\"Vous êtes un évaluateur qui note la pertinence des documents récupérés par rapport à une question utilisateur.\\n\"\n",
    "                 \"Voici les documents récupérés : \\n\\n {documents} \\n\\n\"\n",
    "                 \"Voici la question de l'utilisateur : {question} \\n\"\n",
    "                 \"Si les documents contiennent des mots-clés ou des concepts sémantiques liés à la question, notez-les comme pertinents.\\n\"\n",
    "                 \"Donnez une note binaire, 'oui' ou 'non', pour indiquer si les documents sont pertinents.\",\n",
    "        input_variables=[\"question\", \"documents\"],\n",
    "    )\n",
    "\n",
    "    chain = prompt | structured_llm_grader\n",
    "    docs = last_message.content\n",
    "    response = chain.invoke({\"question\": question, \"documents\": docs})\n",
    "\n",
    "    if response.binary_score == \"oui\":\n",
    "        print(\"Décision : Documents pertinents. Passage à la génération.\")\n",
    "        return \"generate\"\n",
    "    else:\n",
    "        print(\"Décision : Documents non pertinents. Passage à la réécriture.\")\n",
    "        return \"rewrite\"\n",
    "\n",
    "# 5. Nœud `generate` (Générateur de réponse)\n",
    "def generate(state: AgentState):\n",
    "    \"\"\"Génère une réponse finale en utilisant les documents et la question.\"\"\"\n",
    "    print(\"---GÉNÉRATION DE LA RÉPONSE---\")\n",
    "    messages = state[\"messages\"]\n",
    "    question = messages[0].content\n",
    "    last_message = messages[-1]\n",
    "    docs = last_message.content\n",
    "\n",
    "    prompt = f\"Vous êtes un assistant IA spécialisé dans le Machine Learning. Répondez à la question de l'utilisateur en vous basant sur le contexte suivant :\\n\\nContexte : {docs}\\n\\nQuestion : {question}\\n\\nRéponse :\"\n",
    "    \n",
    "    response = llm.invoke(prompt)\n",
    "    return {\"messages\": [response]}\n",
    "\n",
    "# 6. Nœud `rewrite` (Réécrivain de question)\n",
    "def rewrite(state: AgentState):\n",
    "    \"\"\"Reformule la question de l'utilisateur pour une meilleure recherche.\"\"\"\n",
    "    print(\"---RÉÉCRITURE DE LA QUESTION---\")\n",
    "    messages = state[\"messages\"]\n",
    "    question = messages[0].content\n",
    "\n",
    "    prompt = f\"Vous êtes un expert en reformulation de questions. Votre but est d'améliorer la question de l'utilisateur pour la rendre plus spécifique et plus claire pour un moteur de recherche. Ne répondez pas à la question, reformulez-la simplement.\\n\\nQuestion originale : {question}\\n\\nQuestion améliorée :\"\n",
    "    \n",
    "    response = llm.invoke(prompt)\n",
    "    # Nous remplaçons la question originale par la nouvelle pour relancer le cycle.\n",
    "    new_question = HumanMessage(content=response.content)\n",
    "    return {\"messages\": [new_question]}\n",
    "\n",
    "print(\"Tous les nœuds et fonctions du graphe sont définis.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 5 : Assemblage du Graphe avec LangGraph\n",
    "\n",
    "Maintenant que nous avons tous nos composants (état, nœuds), il est temps de les assembler pour former notre agent. C'est ici que la magic de LangGraph opère.\n",
    "\n",
    "1.  **Instanciation du `StateGraph`** : Nous créons une instance de notre graphe en lui passant la structure de notre état, `AgentState`.\n",
    "2.  **Ajout des Nœuds** : Nous déclarons chaque fonction que nous avons définie comme un nœud dans notre graphe, en lui donnant un nom unique.\n",
    "3.  **Définition du Point d'Entrée** : Nous indiquons au graphe par quel nœud le processus doit commencer (`set_entry_point`).\n",
    "4.  **Création des Arêtes Conditionnelles** : C'est la partie la plus importante. Nous connectons les nœuds en définissant des règles. \n",
    "    -   Après le nœud `ai_assistant`, nous utilisons une condition pour vérifier si le LLM a décidé d'appeler un outil. Si oui, on va au nœud `retrieve`. Sinon, le travail est terminé (`END`).\n",
    "    -   Après le nœud `retrieve`, nous utilisons notre fonction `grade_documents` comme aiguilleur. En fonction de sa sortie (\"generate\" ou \"rewrite\"), le flux est dirigé vers le nœud correspondant.\n",
    "5.  **Création des Arêtes Normales** : Nous ajoutons les connexions simples.\n",
    "    -   Après la réécriture (`rewrite`), on boucle en retournant à l'assistant (`ai_assistant`) pour tenter une nouvelle recherche avec la question améliorée. **C'est ce qui crée notre boucle d'auto-correction.**\n",
    "    -   Après la génération (`generate`), le processus est terminé (`END`)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "from langgraph.graph import StateGraph, END\n",
    "\n",
    "# 1. Instanciation du StateGraph\n",
    "workflow = StateGraph(AgentState)\n",
    "\n",
    "# 2. Ajout des Nœuds\n",
    "workflow.add_node(\"My_AI_Assistant\", ai_assistant)\n",
    "workflow.add_node(\"retrieve\", retrieve)\n",
    "workflow.add_node(\"rewrite\", rewrite)\n",
    "workflow.add_node(\"generate\", generate)\n",
    "\n",
    "# 3. Définition du Point d'Entrée\n",
    "workflow.set_entry_point(\"My_AI_Assistant\")\n",
    "\n",
    "# 4. Création des Arêtes Conditionnelles\n",
    "\n",
    "# Condition pour décider si on doit utiliser un outil ou non\n",
    "def should_retrieve(state: AgentState):\n",
    "    print(\"---ROUTAGE : Outil ou Fin ?---\")\n",
    "    messages = state[\"messages\"]\n",
    "    last_message = messages[-1]\n",
    "    # Si le dernier message contient des appels d'outils, alors on doit agir.\n",
    "    if last_message.tool_calls:\n",
    "        print(\"Décision : Appel d'outil détecté. Passage à la recherche.\")\n",
    "        return \"retrieve\"\n",
    "    # Sinon, le LLM a répondu directement, c'est la fin.\n",
    "    print(\"Décision : Pas d'appel d'outil. Fin du processus.\")\n",
    "    return \"end\"\n",
    "\n",
    "workflow.add_conditional_edges(\n",
    "    \"My_AI_Assistant\",\n",
    "    should_retrieve,\n",
    "    {\n",
    "        \"retrieve\": \"retrieve\",\n",
    "        \"end\": END,\n",
    "    },\n",
    ")\n",
    "\n",
    "# Condition pour décider si on génère une réponse ou si on réécrit la question\n",
    "workflow.add_conditional_edges(\n",
    "    \"retrieve\",\n",
    "    grade_documents,\n",
    "    {\n",
    "        \"generate\": \"generate\",\n",
    "        \"rewrite\": \"rewrite\",\n",
    "    },\n",
    ")\n",
    "\n",
    "# 5. Création des Arêtes Normales\n",
    "\n",
    "# Après la réécriture, on boucle en retournant à l'assistant\n",
    "workflow.add_edge(\"rewrite\", \"My_AI_Assistant\")\n",
    "\n",
    "# Après la génération, c'est la fin\n",
    "workflow.add_edge(\"generate\", END)\n",
    "\n",
    "print(\"Graphe assemblé.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 6 : Compilation et Test\n",
    "\n",
    "Notre graphe est maintenant entièrement défini. La dernière étape de construction est de le **compiler**. La compilation transforme notre définition de graphe en un objet `Runnable` que nous pouvons appeler.\n",
    "\n",
    "Nous allons ensuite le tester avec différentes questions pour observer son comportement :\n",
    "-   Une salutation simple (devrait finir directement).\n",
    "-   Une question précise (devrait trouver des documents et générer une réponse).\n",
    "-   Une question vague (devrait déclencher la boucle de réécriture).\n",
    "-   Une question hors sujet (devrait échouer à trouver des documents et potentiellement boucler ou s'arrêter)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compilation du graphe\n",
    "app = workflow.compile()\n",
    "\n",
    "# Visualisation du graphe (nécessite graphviz)\n",
    "# C'est très utile pour vérifier que nos connexions sont correctes.\n",
    "try:\n",
    "    from IPython.display import Image, display\n",
    "    display(Image(app.get_graph().draw_png()))\n",
    "except ImportError:\n",
    "    print(\"Graphviz non installé. Impossible d'afficher le graphe.\")\n",
    "\n",
    "print(\"Graphe compilé et prêt.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- Tests --- \n",
    "\n",
    "import time\n",
    "\n",
    "def run_test(question):\n",
    "    print(f\"\\n--- TEST AVEC LA QUESTION : '{question}' ---\")\n",
    "    inputs = {\"messages\": [HumanMessage(content=question)]}\n",
    "    final_state = app.invoke(inputs)\n",
    "    final_response = final_state['messages'][-1]\n",
    "    print(\"\\n--- RÉPONSE FINALE ---\")\n",
    "    print(final_response.content)\n",
    "    print(\"-------------------------\")\n",
    "\n",
    "# Test 1: Salutation simple\n",
    "# run_test(\"Bonjour, comment ça va ?\")\n",
    "\n",
    "# Test 2: Question précise\n",
    "# run_test(\"Qu'est-ce que le RAG auto-correctif ?\")\n",
    "\n",
    "# Test 3: Question vague\n",
    "# run_test(\"Parle-moi des agents.\")\n",
    "\n",
    "# Test 4: Question hors-sujet\n",
    "# run_test(\"Quelle est la meilleure recette de crêpes ?\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Étape 7 : Fonction d'Interface pour Streamlit\n",
    "\n",
    "Pour que notre application Streamlit puisse utiliser la logique que nous venons de construire, nous devons créer une fonction simple qui sert de pont. \n",
    "\n",
    "Cette fonction, `run_agent`, prendra une chaîne de caractères (la question de l'utilisateur) en entrée et retournera la réponse finale de l'agent. Pour une meilleure expérience utilisateur dans Streamlit, nous allons la transformer en **générateur**. Elle produira la réponse morceau par morceau (`stream`), ce qui permettra d'afficher la réponse au fur et à mesure qu'elle est générée, donnant une impression de \"live typing\"."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import time\n",
    "\n",
    "def run_agent(user_query: str):\n",
    "    \"\"\"\n",
    "    Exécute le graphe de l'agent pour une requête utilisateur donnée et streame la réponse.\n",
    "    \n",
    "    Args:\n",
    "        user_query: La question posée par l'utilisateur.\n",
    "        \n",
    "    Yields:\n",
    "        str: Des morceaux de la réponse finale de l'agent.\n",
    "    \"\"\"\n",
    "    inputs = {\"messages\": [HumanMessage(content=user_query)]}\n",
    "    \n",
    "    # Utiliser .stream() au lieu de .invoke() pour obtenir un générateur\n",
    "    # Cela nous permet de traiter les événements au fur et à mesure qu'ils se produisent dans le graphe.\n",
    "    full_response = \"\"\n",
    "    \n",
    "    # Le stream retourne les états intermédiaires du graphe.\n",
    "    # Nous nous intéressons au dernier message du dernier état.\n",
    "    for output in app.stream(inputs):\n",
    "        # La clé du dictionnaire correspond au nom du noeud qui vient de s'exécuter\n",
    "        for key, value in output.items():\n",
    "            if key == \"__end__\": # La fin du graphe\n",
    "                break\n",
    "            # Nous pouvons ajouter des logs ici pour voir la progression\n",
    "            # print(f\"Output from node '{key}': {value}\")\n",
    "            pass\n",
    "            \n",
    "    # Une fois le stream terminé, l'état final est dans la dernière valeur\n",
    "    final_state = value\n",
    "    if final_state and 'messages' in final_state and final_state['messages']:\n",
    "        final_response_message = final_state['messages'][-1]\n",
    "        full_response = final_response_message.content\n",
    "    else:\n",
    "        full_response = \"Désolé, je n'ai pas pu trouver de réponse.\"\n",
    "\n",
    "    # Simuler un streaming de la réponse finale pour l'affichage\n",
    "    for char in full_response:\n",
    "        yield char\n",
    "        time.sleep(0.01) # Petite pause pour un effet de frappe naturel\n",
    "\n",
    "print(\"La fonction 'run_agent' est prête à être appelée par Streamlit.\")\n",
    "\n",
    "# Exemple d'utilisation de la fonction run_agent\n",
    "# print(\"\\n--- TEST DE LA FONCTION STREAMLIT ---\")\n",
    "# query = \"Explique le concept de 'plan of thought' pour les agents IA.\"\n",
    "# response_generator = run_agent(query)\n",
    "# for chunk in response_generator:\n",
    "#     print(chunk, end=\"\", flush=True)\n",
    "# print()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Challenge : Construire un Agent RAG Auto-Correcteur avec LangGraph\n',
    '\n',
    "Bienvenue dans ce notebook ! L'objectif est de construire un agent conversationnel sophistiqué en utilisant le framework **LangGraph**. \n",
    '\n',
    "Cet agent ne se contentera pas de répondre à des questions sur la base de documents fournis (le principe du RAG - Retrieval-Augmented Generation), mais il sera capable d'évaluer la pertinence des informations récupérées. Si les informations ne sont pas pertinentes, l'agent **reformulera la question** de l'utilisateur pour tenter d'obtenir de meilleurs résultats, créant ainsi une boucle de raisonnement et d'auto-correction.\n",
    '\n',
    '## Plan du Notebook\n',
    '\n',
    "1.  **Configuration de l'Environnement** : Chargement des clés API et des variables d'environnement.\n",
    '2.  **Construction de la Base de Connaissances (Le "R" de RAG)** : Chargement de documents

In [2]:
%pip install python-dotenv langchain-groq langchain-google-genai langgraph langchain-community chromadb